### Setup Python configuration

In [1]:
from mstrio.admin.security_role import SecurityRole, list_security_roles
from mstrio.admin.application import Application
from mstrio.connection import Connection
from mstrio.admin.user import User, list_users
from mstrio.report import Report

In [2]:
username = 'mstr'
password = 'LOBwhz18EmVg'
project_id = 'B7CA92F04B9FAE8D941C3E9B7E0CD754'
project_name = 'MicroStrategy Tutorial'
report_id = 'C9B56A6111EAD7DD3F8D0080EF5538AA'
env_url = 'https://env-181971.customer.cloud.microstrategy.com/MicroStrategyLibrary/api/'
login_mode = 1

### Log in as an administrator

In [3]:
connection = Connection(env_url, username, password, project_id=project_id, login_mode=login_mode)

Connection to MicroStrategy Intelligence Server has been established.


### List all security roles

In [4]:
security_roles = list_security_roles(connection)
['Role name: ' + r.name for r in security_roles]

['Role name: Analyst',
 'Role name: Analytics Architect',
 'Role name: Application Administrator',
 'Role name: Application Architect',
 'Role name: Certifier',
 'Role name: Collaborator',
 'Role name: Consumer',
 'Role name: Customers',
 'Role name: Database Architect',
 'Role name: Embedded Analytics Architect',
 'Role name: IntroBI',
 'Role name: Managers',
 'Role name: Mobile Architect',
 'Role name: new security role',
 'Role name: new_role12',
 'Role name: new_role13',
 'Role name: Normal Users',
 'Role name: Northeast Users',
 'Role name: Platform Administrator',
 'Role name: Power Users',
 'Role name: Project Bulk Administrators',
 'Role name: Project Operations Administrators',
 'Role name: Project Operations Monitors',
 'Role name: Project Resource Settings Administrators',
 'Role name: Project Security Administrators',
 'Role name: System Administrator',
 'Role name: test security role']

### Load set of existing privileges

In [5]:
security_roles[0].fetch()
privileges = [p['name'] for p in security_roles[0].privileges]
['Privilege name: ' + p for p in privileges]

['Privilege name: Create application objects',
 'Privilege name: Create schema objects',
 'Privilege name: Schedule request',
 'Privilege name: Web user',
 'Privilege name: Web create new Report',
 'Privilege name: Web object search',
 'Privilege name: Web save to My Reports',
 'Privilege name: Web advanced drilling',
 'Privilege name: Web export',
 'Privilege name: Web print mode',
 'Privilege name: Web manage Objects',
 'Privilege name: Web save to Shared Reports',
 'Privilege name: Use server cache',
 'Privilege name: Web add to History List',
 'Privilege name: Web change view mode',
 'Privilege name: Web drill and link',
 'Privilege name: Web drill on Metrics',
 'Privilege name: Web format Grid and Graph',
 'Privilege name: Web subscribe to History List',
 'Privilege name: Web simultaneous execution',
 'Privilege name: Web switch Page-By Elements',
 'Privilege name: Web sort',
 'Privilege name: Web filter on selections',
 'Privilege name: Create Derived Metrics',
 'Privilege name: 

### Create a new security role, with the same privileges

In [6]:
security_role = SecurityRole.create(connection,
                                    'test role',
                                    privileges)

SecurityRole object named: 'test role' with ID: '02B82C7E11EAFE5FE7A00080EF950BC8'


### Create a new user

In [7]:
user = User.create(connection,
                   username='testuser',
                   full_name='integration test user1',
                   password='mstr123',
                   require_new_password=False)

Successfully created user named: 'testuser' with ID: '02E8609311EAFE5FE7A00080EF150CC9'


### Assign the new security role to the new user

In [8]:
application = Application(connection, id=project_id)

Application object named: 'MicroStrategy Tutorial' with ID: 'B7CA92F04B9FAE8D941C3E9B7E0CD754'


In [9]:
security_role.grant_to(members=[user.id], application=application)

Granted Security Role 'test role' to ['02E8609311EAFE5FE7A00080EF150CC9']


### Log in as the new user

In [10]:
connection = Connection(base_url=env_url,
                        username=user.username,
                        password='mstr123', 
                        project_id=project_id,
                        login_mode=login_mode)

Connection to MicroStrategy Intelligence Server has been established.


### Verify if the user has access to the report

In [11]:
report = Report(connection,
                report_id=report_id)
report.to_dataframe()

Error getting report contents.
I-Server Error ERR017, (User 'integration test user1' does not have Execute access to the Report Definition object 'C9B56A6111EAD7DD3F8D0080EF5538AA' in Project MicroStrategy Tutorial with ID 'B7CA92F04B9FAE8D941C3E9B7E0CD754'.)
Ticket ID: 4261d24246ff420d9b550629ac0fcbb6



HTTPError: 403 Client Error:  for url: https://env-181971.customer.cloud.microstrategy.com/MicroStrategyLibrary/api/v2/reports/C9B56A6111EAD7DD3F8D0080EF5538AA/instances/?offset=0&limit=1000

As expected accessing the report raises an Exception

### Login back as administrator

In [12]:
connection = Connection(env_url, username, password, project_id=project_id, login_mode=login_mode)

Connection to MicroStrategy Intelligence Server has been established.


### Grant the new user access to a report

In [13]:
user.set_permission('Modify', to_objects=[report_id],
                     object_type=3)

### Log in as the new user

In [15]:
connection = Connection(base_url=env_url,
                        username=user.username,
                        password='mstr123', 
                        project_id=project_id,
                        login_mode=login_mode)

Connection to MicroStrategy Intelligence Server has been established.


### Verify if the user has access to the report

In [16]:
report = Report(connection,
                report_id=report_id)
report.to_dataframe()

,Age Range@DESC,Age Range@ID,Order Count
0,24 and under,1,13311
1,25 to 34,2,25371
2,35 to 44,3,26208
3,45 to 54,4,26247
4,55 and over,5,62853


The user has been granted the permission to use the report

### Login back as administrator

In [17]:
connection = Connection(env_url, username, password, project_id=project_id, login_mode=login_mode)

Connection to MicroStrategy Intelligence Server has been established.


### Find the newly created role ID

In [18]:
role_ids = [r.id for r in list_security_roles(connection, name='test role')]
role_ids

['02B82C7E11EAFE5FE7A00080EF950BC8']

### Delete the newly create role

In [19]:
for r_id in role_ids:
    r = SecurityRole(connection, id = r_id)
    r.delete(force=True)

SecurityRole object named: 'test role' with ID: '02B82C7E11EAFE5FE7A00080EF950BC8'
Deleted security role 02B82C7E11EAFE5FE7A00080EF950BC8.


### Find the newly create user ID

In [20]:
user_ids = [u.id for u in list_users(connection, name='integration test user1')]

In [21]:
user_ids

['02E8609311EAFE5FE7A00080EF150CC9']

### Delete the newly created user

In [22]:
for u_id in user_ids:
    u = User(connection, id=u_id)
    u.delete(force=True)

User object named: 'integration test user1' with ID: '02E8609311EAFE5FE7A00080EF150CC9'
Successfully deleted user 02E8609311EAFE5FE7A00080EF150CC9


## 